In [25]:
import pandas as pd
import matplotlib
import subprocess
import attr
from io import StringIO
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [26]:
import re

# измерить пустой хэндлер

In [27]:
# wrk2 needed

In [28]:
sns.set_style("darkgrid")

def get_benchmark_output(handler: str, rps: int, duration: int = 10):
    command = f'wrk -t1 -c10 -d{duration}s -R{rps} --u_latency'
    command += f' {handler}'
    output = subprocess.getoutput(command)
    return output

def parse_benchmark_output(output: str):
    res = BenchmarkResult()
    res.fill(output)
    return res

def run_benchmark(handler: str, rps: int, duration: int = 10):
    out = get_benchmark_output(handler, rps, duration)
    bench_res = parse_benchmark_output(out)
    return bench_res


def get_percentile_spectrum(out: str) -> pd.DataFrame:

    percentile_spectrum_str = (out.split('Value   Percentile   TotalCount 1/(1-Percentile)')[1].split('#[Mean')[0]).strip()

    percentile_spectrum = pd.read_csv(
        StringIO(percentile_spectrum_str), 
        sep='\s+', 
        header=None, 
        names=['ms', 'percentile', 'total_count', 'x'],
    )
    del percentile_spectrum['x']
    return percentile_spectrum


def plot_spectrum(percentile_spectrum: pd.DataFrame, label: str='experiment', title=''):
    spectrum = percentile_spectrum[percentile_spectrum['percentile'] > .4]
    
    plt.figure(figsize=(13,7));
    plt.xlabel('Percentile')
    plt.ylabel('Response (ms)')
    plt.title('Response spectrum ' + title)
    plt.xticks(np.arange(min(spectrum.percentile), 1.05, .05))
    
    plt.plot(spectrum['percentile'], spectrum['ms'], label=label);
    
    plt.legend()

In [29]:
@attr.s
class BenchmarkResult:
    
    name = attr.ib(default='')
    latency_ms_p95 = attr.ib(default=.0)
    rps = attr.ib(default=.0)
    
    def fill(self, output: str):
        self.rps = float(re.findall(r'Requests/sec:\s+(\d*\.\d*)', output)[0])
        self.spectrum = get_percentile_spectrum(output)
        self.latency_ms_p95 = round(float(self.spectrum[self.spectrum['percentile'] > .949].iloc[0].ms), 3)
        
    def plot(self):
        plot_spectrum(self.spectrum, label=self.name, title=self.name)

In [30]:
class Benchmark:
    
    def __init__(self, handler='_info', host='http://127.0.0.1:8890'):
        self.handler = handler
        self.host = host
        self.results = []
        
    def run(self, rps: list, duration = 120):
        results = []
        for rate in rps:
            res = BenchmarkResult(name=f'{self.handler} {rate} rps')
            out = get_benchmark_output(self.host + '/' + self.handler, rps=rate, duration=duration)
            res.fill(out)
            print(res)
            results.append(res)
        self.resuts = results
        return results

In [31]:
b0 = Benchmark(host='http://127.0.0.1:8890', handler='_info')
results = b0.run(rps=[10, 100, 500], duration=2)
# print(results)

BenchmarkResult(name='_info 10 rps', latency_ms_p95=2.179, rps=10.49)
BenchmarkResult(name='_info 100 rps', latency_ms_p95=2.201, rps=100.44)
BenchmarkResult(name='_info 500 rps', latency_ms_p95=2.699, rps=496.28)


In [32]:
b1 = Benchmark(host='http://127.0.0.1:8890', handler='sleep50')
results = b1.run(rps=[50, 100, 200], duration=10)
# print(results)

BenchmarkResult(name='sleep50 50 rps', latency_ms_p95=2.791, rps=50.09)
BenchmarkResult(name='sleep50 100 rps', latency_ms_p95=2.655, rps=100.08)
BenchmarkResult(name='sleep50 200 rps', latency_ms_p95=2.505, rps=200.05)


In [ ]:
b2 = Benchmark(host='http://127.0.0.1:8890', handler='sleep100')
results = b2.run(rps=[50, 100, 200], duration=10)
# print(results)

In [ ]:
b3 = Benchmark(host='http://127.0.0.1:8890', handler='sleep100')
results = b2.run(rps=[300, 400, 500], duration=10)
# print(results)